In [ ]:
import pandas as pd
import numpy as np
import glob
import xgboost as xgb
import requests
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# URL to the GitHub folder (NOT raw, NOT the file, but the folder)
api_url = "https://api.github.com/HardyUnine/ML_Project/tree/main/plan_b"

# Get list of files in the folder
files = requests.get(api_url).json()

# Extract only CSV file URLs
csv_urls = [f["download_url"] for f in files if f["name"].endswith(".csv")]

# Load all CSVs into one dataframe
dfs = []
for url in csv_urls:
    df = pd.read_csv(url)
    df["source_file"] = url.split("/")[-1].replace(".csv", "")  # optional tag
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)


# df = pd.read_csv("nom_a_changer.csv") # CHANGE CSV NAME HERE ZZ
# df.head()

# # Cant have white space between words when wanting to draw trees!!

[]


ValueError: No objects to concatenate

In [ ]:

# Split data in dependent and independant variables
X = df("if yes or no there was a short squeeze", axis=1).copy # REPLACE NAME ZZ
X.head()

y = df["if yes or no there was a short squeeze"].copy() # REPLACE NAME ZZ
y.head()

In [ ]:
sum(y)/len(y) # to check how many positives we have
# sum of how many squeezes, divided by total 
# data will probably be inbalances (ex. 30% squeezes) so need to make sure that there is 30% in both the train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y) # stratify = yes, we want same percetages everywjere

sum(y_train)/len(y_train)
sum(y_test)/len(y_test) # check that they have the same percentages of squeezes

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=None) # creates shell in which we will make the trees
clf_xgb.fit(X_train, 
            y_train, 
            verbose=True, # verbose="tell me what you are doing when you are doing it"
            early_stopping_rounds=10, # if after 10 trees the prediction is not improving, stop
            eval_metric="aucpr",
            eval_set=[(X_test, y_test)]) # it is testing how many trees with need with the test df